# Talking about the DataSet
The data set was downloaded from [here](http://deeplearning.net/data/mnist/)
And this dataset contains images of size 28x28 and have been flattened to a 784 row.

Now we need to reshape the data to a 2d array for that we will be using numpy 

We will also be using the matplotlib to show the data.


In PyTorch when you give and input or get some output it completely has to be in the form of a tensor.

But what is a TENSOR? And why is is used in the Machine Learning community so extensivelt?

Okay, so when we were studying highschool physics we learned about scalar, vector and tensors. The tensor we are talking about now and the one there are completly the same.

Since the data we have are in the form of Numpy arrays we need to convert them to tensors

In [26]:
import pickle
import gzip
from matplotlib import pyplot as plot
import numpy as np
import torch
from pathlib import Path
import requests

DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)

with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
        
# plot.imshow(x_train[0].reshape((28,28)), cmap='gray')
# print(x_train.shape)

x_train, y_train, x_valid, y_valid = map(torch.tensor, (x_train, y_train, x_valid, y_valid))
n, c = x_train.shape

# x_train, x_train.shape, y_train.min(), y_train.max()
# print(x_train, y_train)
# print(x_train.shape)
# print(y_train.min(), y_train.max())

# Implementing the Neural Network

We will be implementing the neural network from scratch using the math library and the going to the pytorch implemention.

## The gradient decent

Before getting started we should first talk about gradient decent. In PyTorch every tensor has a flag `requires_grad` which is set to false by default. But can be set to true. It records all the operation performed on the data and also apparently helps in gradient-decent.

## Initialising the weights

Here the weights are brinh initialised using xavier distibution which is `random_numbers/sq-root(total-numbers-values`. And the `requires_flag` has also been set to `true`. 

## Activation Function
Here the `log_softmaxx` function has been used.
log_softmax = `log[ exp(x) / sum(exp(xi))]`  


In [27]:
import math

weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)

def model(xb):
    return log_softmax(xb @ weights + bias)

bs = 64  # batch size

xb = x_train[0:bs]  # a mini-batch from x
preds = model(xb)  # predictions
preds[0], preds.shape
print(preds[0], preds.shape)

def nll(input, target):
    return -input[range(target.shape[0]), target].mean()

loss_func = nll

yb = y_train[0:bs]
print(loss_func(preds, yb))

def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

tensor([-2.4702, -2.6971, -2.2900, -1.9528, -2.3051, -1.9358, -2.1061, -2.1695,
        -2.6190, -2.9564], grad_fn=<SelectBackward>) torch.Size([64, 10])
tensor(2.3507, grad_fn=<NegBackward>)


In [28]:
print(accuracy(preds, yb))


tensor(0.1250)


In [29]:
from IPython.core.debugger import set_trace

lr = 0.5  # learning rate
epochs = 2  # how many epochs to train for

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        #         set_trace()
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        pred = model(xb)
        loss = loss_func(pred, yb)

        loss.backward()
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()
            
print(loss_func(model(xb), yb), accuracy(model(xb), yb))


tensor(0.0825, grad_fn=<NegBackward>) tensor(1.)


_____________________________________________________________________________________________

_______________________________________________________________________________________________

In [30]:
import torch.nn.functional as F

from torch import nn


loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias

class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(784,10)/math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))
        
    def forword(self, xb):
        return xb @ self.weights + self.bias


model = Mnist_Logistic()


In [31]:
print(loss_func(model(xb), yb))

NotImplementedError: 